In [1]:
# The directory for top level folder
dir_ = "/home/sugam/Work/20-29 Deep Learning/22 Projects/Optimization of Energy Using AIML/data/"

In [2]:
raw_data_dir = dir_ + 'raw/'
processed_data_dir = dir_+'Processed/'

# 1. MAIN SETUP

In [3]:
# Genderal imports 
import numpy as np
import pandas as pd
import os, sys, gc,time,warnings,pickle,psutil,random
from math import ceil
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [4]:
############################# MEMORY PROFILER
## Displays memory used by dataframe

def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [5]:
########################## MEMORY REDUCER
## Function which checks each column and manage the dtype automatically


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print(f'Mem. usage decreased to {end_mem:5.2f} Mb {(100 * (start_mem - end_mem) / start_mem):.1f}% reduction')
    return df

In [6]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1,df2,merge_on):
    merged_df = df1[merge_on] # merge_on is multiple columns
    merged_df = merged_df.merge(df2,on=merge_on,how='left')
    new_columns = [col for col in list(merged_df) if col not in merge_on]
    df1 = pd.concat([df1,merged_df[new_columns]],axis=1)
    return df1

In [7]:
######################### Vars
###################################################################
TARGET = ["Load_NW","Load_SW"] # main target ->  Total Load of North Wing (Load_NW), Total Load of South Wing (Load_SW)
END_TRAIN = '2020-12-04 23:45:00' # Last day of train set
MAIN_INDEX = ['date'] # Identify each entry by these columns

# 2. PART -1 

In [8]:
####################### LOAD DATA
##################################################################
print("⏳ Load Main Data")

# Refering our data without any modification and dtype
train_df = pd.read_csv(processed_data_dir+'merged_file.csv',
                       header=1,
                       index_col=0,
                       parse_dates = True)

train_df.drop("Unnamed: 6", 
              axis=1,
              inplace=True) # Garbage column

#train_df.drop_duplicates(subset='date',inplace=True)

train_df.index = pd.to_datetime(train_df.index,format='mixed',dayfirst=True) # Converting index to date time format

print("✅ Load Main Data")

⏳ Load Main Data
✅ Load Main Data


In [9]:
train_df.shape

(64672, 114)

In [10]:
print("Checking if the index is unique or not : ",str(len(pd.unique(train_df.index))) == train_df.shape[0])

Checking if the index is unique or not :  False


In [11]:
train_df = train_df[~train_df.duplicated()]

In [12]:
# Combining all North wings and South wing load into a single variables

train_df["Load_SW"] = (train_df['Miscellaneous electric load for the South Wing']+
                       train_df['Lighting load for the South Wing']+
                       train_df['Heating Ventilation and Air Conditioning load for the South Wing']
                      )

train_df.drop(['Miscellaneous electric load for the South Wing',
               'Lighting load for the South Wing',
               'Heating Ventilation and Air Conditioning load for the South Wing']
              ,axis=1,
             inplace=True)

train_df["Load_NW"] = (train_df['Miscellaneous electric load for the North Wing']+
                       train_df['Heating Ventilation and Air Conditioning load for the Nouth Wing']
                      )

train_df.drop(['Miscellaneous electric load for the North Wing',
               'Heating Ventilation and Air Conditioning load for the Nouth Wing']
              ,axis=1,
             inplace=True)

In [27]:
print(train_df.isnull().sum(axis=0)[train_df.isnull().sum(axis=0)>0])

Series([], dtype: int64)


In [14]:
na_columns = train_df.isnull().sum(axis=0)[train_df.isnull().sum(axis=0)>0].index

In [15]:
train_df.fillna("bfill",inplace=True)

In [16]:
# # Creating month and year columns
# train_df["year"] = train_df.index.year
# train_df["month"] = train_df.index.month

In [17]:
# for col in na_columns:
#     temp = pd.DataFrame()
#     temp = train_df.groupby(["year","month"])[col].agg('mean').reset_index()
#     temp.rename(columns={col: "mean_column"},inplace=True)
#     temp = pd.merge(train_df,temp,how="left",on=["year","month"])
#     temp.loc[temp[col].isna(),[col]] = temp["mean_column"]

# temp.isna().sum(axis=0)[temp.isna().sum(axis=0)>0]
# #train_df[na_columns] = temp[na_columns]

In [18]:
# temp = pd.DataFrame()
# for col in na_columns[:]:
#     temp = train_df.groupby(["year","month"])[col].agg('mean').reset_index()
#     temp.rename(columns={col: "mean_column"},inplace=True)
#     temp = pd.merge(train_df,temp,how="left",on=["year","month"])
#     temp.loc[temp[col].isna(),col] = temp["mean_column"]
#     print(temp[col].isna().sum())
# temp.isna().sum(axis=0)
# train_df[na_columns] = temp[na_columns]
# train_df.isna().sum(axis=0)[train_df.isna().sum(axis=0)>0]

In [ ]:
# In order to make predictions, it's essential to incorporate the test set into our dataframe. 
# The following code is responsible for appending new rows to accommodate future data.
# We are going to predict the probable data for next 24 hour(in future) in time interval of 15 min
# 24*60 (minutes in a day) / 15 = 96 (15's in 24 hour)
for i in range(1,97):
    temp_df = train_df.copy()
    temp_df


In [28]:
train_df.tail()

,Heat pump heating water supply temperature,Roof Top Unit * supply air temperature setpoint_001,Roof Top Unit * supply air temperature setpoint_002,Roof Top Unit * supply air temperature setpoint_003,Roof Top Unit * supply air temperature setpoint_004,Roof Top Unit * supply air temperature_001,Roof Top Unit * supply air temperature_002,Roof Top Unit * supply air temperature_003,Roof Top Unit * supply air temperature_004,Roof Top Unit * return air temperature_001,...,Zone temperature of exterior zone065,Zone temperature of exterior zone066,Zone temperature of exterior zone067,Zone temperature of exterior zone068,Zone temperature of exterior zone069,Zone temperature of exterior zone070,Zone temperature of exterior zone071,Zone temperature of exterior zone072,Load_SW,Load_NW
date,,,,,,,,,,,,,,,,,,,,,
2020-02-05 10:45:00,123.2,68.0,68.0,68.0,68.0,68.9,69.0,67.4,67.7,69.6,...,69.7,68.8,70.9,69.7,70.0,69.5,69.5,71.0,24.434211,9.77806
2020-02-05 11:00:00,123.4,68.0,68.0,68.0,68.0,68.8,67.9,66.4,67.7,69.5,...,69.7,68.8,70.8,69.7,69.9,69.5,69.4,71.0,23.813333,8.981637
2020-02-05 11:15:00,124.0,68.0,68.0,68.0,68.0,68.8,68.5,65.2,67.5,69.5,...,69.7,68.7,70.8,69.5,69.9,69.5,69.4,71.0,25.387218,8.208946
2020-02-05 11:30:00,122.9,68.0,68.0,68.0,68.0,68.9,68.0,66.1,67.2,69.3,...,69.6,68.7,70.7,69.5,69.8,69.4,69.3,71.0,24.8375,8.798963
2020-02-05 11:45:00,122.9,68.0,68.0,68.0,68.0,68.9,68.8,67.1,67.4,69.3,...,69.6,68.6,70.6,69.5,69.8,69.4,69.2,71.0,25.025,9.61075


In [31]:
train_df.index[-1]+ pd.Timedelta(minutes=15)

Timestamp('2020-02-05 12:00:00')

In [19]:
# ####################### MAKE GRID
# ##################################################################
# print("Create Grid")

# # We are unpivoting the table inorder to convert wide format table into long format
# index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
# grid_df = pd.melt(train_df,
#                  id_vars = index_columns,
#                  var_name = 'd',
#                  value_name = TARGET)
# # In the train_df, there are very few training rows
# # But each day can provide a lot of training data
# print(f"Train rows: {len(train_df)} --- {len(grid_df)}")

# # To be able to make predictions we need to add test set to our grid
# # Below code adds the new rows for future data.
# # It will add 28 days in the future
# add_grid = pd.DataFrame()
# for i in range(1,29):
#     temp_df = train_df[index_columns]
#     temp_df = temp_df.drop_duplicates() # Ensures data is unique which you going to predict in the future
#     temp_df['d'] = "d_"+str(END_TRAIN+i)
#     temp_df[TARGET] = np.nan
#     add_grid = pd.concat([add_grid,temp_df])

# grid_df = pd.concat([grid_df,add_grid])
# grid_df.reset_index(drop=False,inplace=True)
# del temp_df, add_grid
# del train_df

# # Let's check our memory usage
# print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
# # We can free some memory 
# # by converting "strings" to categorical
# # it will not affect merging and 
# # we will not lose any valuable data
# for col in index_columns:
#     grid_df[col] = grid_df[col].astype('category')

# # Let's check again memory usage
# print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

In [20]:
# ########################### Product Release date
# #################################################################################
# print('Release week')

# # It seems that leadings zero values
# # in each train_df item row
# # are not real 0 sales but mean
# # absence for the item in the store
# # we can safe some memory by removing
# # such zeros

# # Prices are set by week
# # so it we will have not very accurate release week 
# release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
# release_df.columns = ['store_id','item_id','release']

# # Now we can merge release_df
# grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
# del release_df

# # We want to remove some "zeros" rows
# # from grid_df 
# # to do it we need wm_yr_wk column
# # let's merge partly calendar_df to have it
# grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])
                      
# # Now we can cutoff some rows 
# # and safe memory 
# grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
# grid_df = grid_df.reset_index(drop=True)

# # Let's check our memory usage
# print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# # Should we keep release week 
# # as one of the features?
# # Only good CV can give the answer.
# # Let's minify the release values.
# # Min transformation will not help here 
# # as int16 -> Integer (-32768 to 32767)
# # and our grid_df['release'].max() serves for int16
# # but we have have an idea how to transform other columns in case we will need it
# grid_df['release'] = grid_df['release'] - grid_df['release'].min()
# grid_df['release'] = grid_df['release'].astype(np.int16)

# # Let's check again memory usage
# print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))


In [21]:


# ########################### Save part 1
# #################################################################################
# print('Save Part 1')

# # We have our BASE grid ready
# # and can save it as pickle file
# # for future use (model training)
# grid_df.to_pickle(processed_data_dir+'grid_part_1.pkl')
# print('Size:', grid_df.shape)

In [22]:
# ########################## PRICES
# ######################################################
# print("Prices")

# # We can do some basic aggregration
# prices_df['price_max'] = prices_df.groupby(["store_id","item_id"])["sell_price"].transform("max") # this gives you the maximum price each unique product has been sold to .
# # Same product can be sold to many prices but the above code finds out the maximum price. Finally it replaces each item in that group with the maximum value.
# prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
# prices_df["price_mean"] = prices_df.groupby(["store_id","item_id"])['sell_price'].transform("mean")
# prices_df["price_std"] = prices_df.groupby(["store_id","item_id"])['sell_price'].transform("std")


# # Doing price normalization(min/max scaling)
# prices_df["price_norm"] = prices_df["sell_price"]/prices_df["price_max"]


In [23]:
# # Some items can be inflation dependent and some can be stable
# prices_df["price_nunique"] = prices_df.groupby(["store_id","item_id"])["sell_price"].transform("nunique")
# prices_df["item_nuinque"] = prices_df.groupby(["store_id","item_id"])["item_id"].transform("nunique")

In [24]:
# # Making rolling aggregations but with months and years as window
# calendar_prices = calendar_df[["wm_yr_wk","month","year"]]
# calendar_prices = calendar_prices.drop_duplicates(subset=["wm_yr_wk"])
# prices_df = prices_df.merge(calendar_prices[["wm_yr_wk","month","year"]],on=["wm_yr_wk"],how="left")

In [25]:
# del calendar_prices

In [26]:
# # Now we can add price "momentum" (some sort of)
# # Shifted by week 
# # by month mean
# # by year mean
# prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
# prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
# prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

# del prices_df['month'], prices_df['year']